In [ ]:
import requests
import json
import os
from datetime import datetime

# Configuração
BACKEND_URL = "http://localhost:3001"
API_URL = f"{BACKEND_URL}/api"

def print_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

print(f"✅ Configurado para testar: {BACKEND_URL}")

## 1. Health Check
Verifica se o backend está online e respondendo.

In [ ]:
try:
    response = requests.get(f"{BACKEND_URL}/health")
    print(f"Status Code: {response.status_code}")
    print_json(response.json())
except Exception as e:
    print(f"❌ Erro ao conectar: {e}")

## 2. Teste de IA (Gemini)
Envia um prompt para o endpoint de streaming da IA. Nota: Este teste usa o endpoint REST padrão, não o streaming SSE, para simplificar a visualização.

In [ ]:
payload = {
    "messages": [
        {"role": "system", "content": "Você é um assistente jurídico útil."},
        {"role": "user", "content": "Resuma o que é um Habeas Corpus em uma frase."}
    ],
    "temperature": 0.7
}

print("🤖 Enviando requisição para IA...")
try:
    # Nota: O endpoint /api/llm-stream retorna SSE (Server-Sent Events).
    # Aqui vamos ler o stream e montar a resposta.
    with requests.post(f"{API_URL}/llm-stream", json=payload, stream=True) as r:
        print(f"Status: {r.status_code}")
        full_response = ""
        for line in r.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                if decoded_line.startswith('data: '):
                    data_str = decoded_line[6:]
                    try:
                        data_json = json.loads(data_str)
                        if data_json.get('type') == 'content':
                            content = data_json.get('content', '')
                            print(content, end='', flush=True)
                            full_response += content
                        elif data_json.get('type') == 'error':
                            print(f"\n❌ Erro da IA: {data_json.get('message')}")
                    except:
                        pass
    print("\n\n✅ Resposta completa recebida.")
except Exception as e:
    print(f"❌ Erro: {e}")

## 3. Teste KV Store (Redis/Memória)
Testa a persistência de dados.

In [ ]:
# 1. Salvar um valor
test_key = "teste-notebook"
test_value = {"data": str(datetime.now()), "autor": "Notebook Test"}

print(f"💾 Salvando chave '{test_key}'...")
res_post = requests.post(f"{API_URL}/kv/{test_key}", json={"value": test_value})
print_json(res_post.json())

# 2. Recuperar o valor
print(f"\n📂 Recuperando chave '{test_key}'...")
res_get = requests.get(f"{API_URL}/kv/{test_key}")
print_json(res_get.json())

## 4. Listar Agentes
Verifica quais agentes estão disponíveis no sistema.

In [ ]:
try:
    response = requests.get(f"{API_URL}/agents/list")
    if response.status_code == 200:
        agents = response.json()
        print(f"✅ Encontrados {len(agents)} agentes:")
        for agent in agents:
            print(f" - {agent.get('name')} ({agent.get('id')})")
    else:
        print(f"⚠️ Status inesperado: {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"❌ Erro: {e}")